In [30]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from sklearn.metrics.pairwise import cosine_similarity
import cv2

In [31]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))


Running on device: cuda:0


In [32]:
mtcnn = MTCNN(image_size=160, margin=0, min_face_size=20,
              thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
              device=device)

In [33]:
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)


In [34]:
preprocess = transforms.Compose([
    transforms.Resize((160, 160)),      # Resize ảnh về 160x160
    transforms.ToTensor(),              # Chuyển ảnh thành tensor
    transforms.Normalize(               # Chuẩn hóa giá trị pixel
        mean=[0.5, 0.5, 0.5], 
        std=[0.5, 0.5, 0.5]
    )
])

In [37]:
folder_path = "Data"
image_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(('.jpg', '.png'))]

print("Danh sách ảnh:", image_paths)

Danh sách ảnh: ['Data/ac84e587-7413-11ef-bf8a-e81f155e7428.jpg', 'Data/4a00b567-e124-4b28-89f1-36d331729d12.jpg', 'Data/583448a4-aad6-11ef-b52d-c403a8a5aea1.jpg', 'Data/caebf623-1951-43ed-99e0-dd76e1c9569e.jpg', 'Data/a7b937a7-7413-11ef-b46f-e81f155e7428.jpg', 'Data/ab16ce1e-7413-11ef-a7e1-e81f155e7428.jpg', 'Data/a6a96299-7413-11ef-8665-e81f155e7428.jpg', 'Data/b09b68ad-0441-4f77-a477-1eb8c900222c.jpg', 'Data/f58342e0-cc56-429f-aa99-aca8b838899e.jpg', 'Data/579baa43-8319-4e8d-9c45-c90bd6c9c3f9.jpg']


In [38]:
def extract_embedding(model, image_path, device):
    image = Image.open(image_path).convert("RGB")
    input_tensor = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model(input_tensor)
    return embedding.cpu().numpy()

In [39]:
embeddings = {}
for file_name in os.listdir(folder_path):
    if file_name.endswith(('.jpg', '.png')):  # Chỉ lấy ảnh có đuôi .jpg hoặc .png
        # Tải ảnh và tiền xử lý
        image_path = os.path.join(folder_path, file_name)
        image = Image.open(image_path).convert("RGB")
        input_tensor = preprocess(image).unsqueeze(0).to(device)

        # Trích xuất embedding
        with torch.no_grad():
            embedding = model(input_tensor).cpu().numpy()

        # Lưu embedding theo tên file
        person_name = os.path.splitext(file_name)[0]  # Lấy tên file mà không có phần mở rộng
        embeddings[person_name] = embedding

# Lưu embeddings vào file để tái sử dụng
np.save("reference_embeddings.npy", embeddings)
print("Embedding của các ảnh tham chiếu đã được lưu.")

Embedding của các ảnh tham chiếu đã được lưu.


In [40]:
reference_embeddings = np.load("reference_embeddings.npy", allow_pickle=True).item()
for name, embedding in reference_embeddings.items():
    if not isinstance(embedding, np.ndarray):  # Kiểm tra nếu không phải NumPy array
        reference_embeddings[name] = np.array(embedding)

        
def cosine_similarity(embedding1, embedding2):
    # Đảm bảo các embedding là NumPy array
    if isinstance(embedding1, dict) or isinstance(embedding2, dict):
        raise ValueError("Input embeddings must be NumPy arrays, not dictionaries.")
    if not isinstance(embedding1, np.ndarray) or not isinstance(embedding2, np.ndarray):
        raise TypeError("Both embeddings must be NumPy arrays.")

    # Tính độ tương đồng cosine
    return np.dot(embedding1, embedding2.T) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

In [41]:
# Mở camera
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Không thể truy cập camera.")
        break
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    box, prob = mtcnn.detect(rgb_frame)

    if box is not None:
        for (x1, y1, x2, y2) in box:
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            face = rgb_frame[y1:y2, x1:x2]  # Cắt khuôn mặt
            face_image = Image.fromarray(face).resize((160, 160))

            # Trích xuất embedding từ khuôn mặt live cam
            if cv2.waitKey(1) & 0xFF == ord("s"):
                # Lưu ảnh
                saved_path = "captured_face.jpg"
                face_image.save(saved_path)
                print(f"Ảnh đã được lưu: {saved_path}")

                # Tính embedding từ ảnh chụp
                face_tensor = preprocess(face_image).unsqueeze(0).to(device)  # Xử lý ảnh
                with torch.no_grad():
                    face_embedding = model(face_tensor).cpu().numpy()  # Tính embedding

                # So sánh với embedding tham chiếu
                similarity = cosine_similarity(face_embedding, embedding)[0][0]
                print(f"Độ tương đồng: {similarity:.2f}")

                # Kiểm tra ngưỡng tương đồng
                if similarity > 0.8:  # Ngưỡng tùy chỉnh
                    print("Khuôn mặt khớp với ảnh tham chiếu!")
                else:
                    print("Khuôn mặt không khớp với ảnh tham chiếu!")
    cv2.imshow('Live Face Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Ảnh đã được lưu: captured_face.jpg
Độ tương đồng: 0.68
Khuôn mặt không khớp với ảnh tham chiếu!
Ảnh đã được lưu: captured_face.jpg
Độ tương đồng: 0.61
Khuôn mặt không khớp với ảnh tham chiếu!
